In [115]:
import serial
import time
import numpy as np

#pyqtgraph -> fast plotting
import pyqtgraph as pg
from pyqtgraph.Qt import QtGui

#matplotlib -> normal plotting
import matplotlib.pyplot as plt
%matplotlib inline

import copy

In [116]:
# Change the configuration file name
configFileName = 'mmw_pplcount_demo_default.cfg'

global CLIport
global Dataport

CLIport = {}
Dataport = {}

CLIport = serial.Serial('COM4', 115200)
if not(CLIport.is_open):
    CLIport.open()
Dataport = serial.Serial('COM3', 921600)
if not(Dataport.is_open):
    Dataport.open()

In [117]:
# Read the configuration file and send it to the board
config = [line.rstrip('\r\n') for line in open(configFileName)]
for i in config:
    CLIport.write((i+'\n').encode())
    print(i)
    time.sleep(0.01)

#close control port
CLIport.close()

dfeDataOutputMode 1
channelCfg 15 3 0
adcCfg 2 1
adcbufCfg 0 1 1 1 
profileCfg 0 77 30 7 62 0 0 60 1 128 2500 0 0 30
chirpCfg 0 0 0 0 0 0 0 1
chirpCfg 1 1 0 0 0 0 0 2
frameCfg 0 1 128 0 50 1 0
lowPower 0 1
guiMonitor 1 1 0 0
cfarCfg 6 4 4 4 4 16 16 4 4 50 62 0
doaCfg 600 1875 30 1
SceneryParam -6 6 0.05 6
GatingParam 4 3 2 0
StateParam 10 5 10 100 5
AllocationParam 450 0.01 25 1 2
VariationParam 0.289 0.289 1.0
PointCloudEn 1
trackingCfg 1 2 250 20 200 50 90
sensorStart


In [118]:
#initialise variables
lostSync = False

#valid header variables and constant
magicBytes = np.array([2,1,4,3,6,5,8,7], dtype= 'uint8')

isMagicOk = False
isDataOk = False
gotHeader = False

frameHeaderLength = 52 #52 bytes long
tlvHeaderLengthInBytes = 8
pointLengthInBytes = 16
frameNumber = 1
targetFrameNumber = 0



In [119]:
def validateChecksum(recieveHeader):
    uint32Convertor = [1, 2**8, 2**16, 2**24]
    uint16Convertor = uint32Convertor[0:2]
    h = np.uint16(recieveHeader)
    a = np.uint32(sum(h))
    b = np.uint16(a)
    CS = np.uint16(~(b))
    return CS

In [120]:
def readHeader(recieveHeader):
    headerContent = dict()
    index = 0
    
    headerContent['magicBytes'] = recieveHeader[index:index+8]
    index += 20
    
    headerContent['packetLength'] = recieveHeader[index:index+4].view(dtype=np.uint32)
    index += 4
        
    headerContent['frameNumber'] = recieveHeader[index:index+4].view(dtype=np.uint32)
    index += 24
    
    headerContent['numTLVs'] = recieveHeader[index:index+2].view(dtype=np.uint16)
    
    return headerContent

In [129]:
def tlvParsing(data, dataLength, tlvHeaderLengthInBytes, pointLengthInBytes):
    
    
    index = 0
    #tlv header parsing
    tlvType = data[index:index+4].view(dtype=np.uint32)
    tlvLength = data[index+4:index+8].view(dtype=np.uint32)
    
    #TLV size check
    print(tlvLength + index)
    print(dataLength)
    print(tlvLength + index > dataLength)
    if (tlvLength + index > dataLength):
        print('TLV SIZE IS WRONG')
        lostSync = True
        return
    
    index += tlvHeaderLengthInBytes
    tlvDataLength = tlvLength - tlvHeaderLengthInBytes
    if tlvType == 6: #point cloud TLV
        numberOfPoints = tlvDataLength/pointLengthInBytes
        print('NUMBER OF POINTS ', str(numberOfPoints))
        if numberOfPoints > 0:
           
            p = data[index:index+tlvDataLength]
            print(p)
            #form the appropriate array 
            #each point is 16 bytes - 4 bytes for each property - range, azimuth, doppler, snr
            
                
            
            return p
        
    

In [130]:
#read and parse data

while Dataport.is_open:
    
    while (not(lostSync) and Dataport.is_open):
        #check for a valid frame header
        if not(gotHeader):
            
            #in_waiting = amount of bytes in the buffer
            rawRecieveHeader = Dataport.read(frameHeaderLength)
            recieveHeader = np.frombuffer(rawRecieveHeader, dtype = 'uint8')
            
        #magic byte check
        if not(np.array_equal(recieveHeader[0:8],magicBytes)):
            print('NO SYNC PATTERN')
            lostSync = True
            break

        #valid the checksum
#         CS = validateChecksum(recieveHeader)
        
        #have a valid frame header
        headerContent = readHeader(recieveHeader)
        print(headerContent)
        print('DONE')
        
        if (gotHeader):
            if headerContent['frameNumber'] > targetFrameNumber:
                targetFrameNumber = headerContent['frameNumber']
                gotHeader = False
                print('FOUND SYNC AT FRAME NUMBER ' + str(targetFrameNumber))
            else:
                print('OLD FRAME')
                gotHeader = False
                lostSync = True
                break
                
        dataLength = int(headerContent['packetLength'] - frameHeaderLength)
        if dataLength > 0:
            #read the rest of the packet
            rawData = Dataport.read(dataLength)
            data = np.frombuffer(rawData, dtype = 'uint8')
            pointCloud = tlvParsing(data, dataLength, tlvHeaderLengthInBytes, pointLengthInBytes)
            print(pointCloud)
            
            
            
            

{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8), 'packetLength': array([330], dtype=uint32), 'frameNumber': array([419887], dtype=uint32), 'numTLVs': array([3], dtype=uint16)}
DONE
[120]
278
[False]
NUMBER OF POINTS  [7.]


TypeError: only integer scalar arrays can be converted to a scalar index

In [85]:
print(pointCloud)

None


In [98]:
x = np.array([89,153,34,63], dtype=np.uint8)
print(x.view(dtype=np.single))

[0.6351524]
